In [6]:
import torch
from transformers import pipeline
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
pipe = pipeline(
    "text-generation",
    model=model_id,
    device = 0
)

c:\Users\Yl\miniconda3\envs\cuda_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

torch.cuda.empty_cache()
prompt = '["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'
model_a_response = '["As an AI, I don\'t have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decision-making. Critics may argue that positions should be based purely on merit, and that quotas could potentially lead to tokenism or unqualified candidates being promoted. Morality can be subjective and differs from person to person based on their personal beliefs, cultural background, and values. It\'s a topic with valid arguments on both sides, and it\'s up to each individual or organization to decide what they believe is right.","As an AI, I don\'t eat, so I don\'t have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of the fruit has no place on a pizza. It\'s a pizza puzzle that might never be solved. So, whether pineapple belongs on a pizza or not, it truly depends on your taste buds!"]'
model_b_response = '["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\\n\\nHere are some arguments in favor of and against such policies:\\n\\n**Arguments in favor:**\\n\\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\\n\\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\\n\\n3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers.\\n\\n4. **Role Modeling:** Increased visibility of female leaders can inspire and encourage other women and girls to pursue leadership roles.\\n\\n**Arguments against:**\\n\\n1. **Reverse Discrimination:** Setting a quota for female representation might lead to the perception or reality of reverse discrimination, where men might be overlooked for positions despite being qualified, simply to meet the gender target.\\n\\n2. **Meritocracy Concerns:** Critics argue that employment and promotion should be based solely on merit and qualifications, not gender. They fear that quotas could compromise the quality of leadership if less qualified individuals are promoted to meet gender targets.\\n\\n3. **Tokenism:** There is a risk that women appointed to meet quotas might be seen as \\"tokens,\\" undermining their legitimacy and potentially leading to resentment among colleagues.\\n\\n4. **Oversimplification of Diversity:** Focusing only on gender might overlook other important aspects of diversity, such as race, ethnicity, socioeconomic background, or sexual orientation.\\n\\nUltimately, the morality of aiming for a certain percentage of females in managerial positions depends on one\'s ethical framework and the context in which such policies are implemented. Proponents of gender diversity targets often argue that these measures are necessary as a transitional mechanism to create a level playing field, while opponents may argue for a \\"color-blind\\" or \\"gender-blind\\" approach to hiring and promotions.\\n\\nOrganizations and societies must carefully consider the implications of these strategies and strive to implement policies that promote fairness and equality while minimizing negative consequences. It is also important to address the root causes of gender inequality, such as unconscious bias, work-life balance challenges, and lack of supportive policies, to create an environment where everyone has the opportunity to succeed based on their capabilities and contributions.","Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest.\\n\\nLet\'s slice into the debate with a zest of fun:\\n\\n**Team Pineapple:** \\"Absolutely, yes! Pineapple on pizza is like a beach party in your mouth. The sweet juiciness of pineapple chunks frolicking with savory ham or bacon creates a flavor wave that surfs across the cheesy ocean of deliciousness. It\'s the Mardi Gras of pizzas, where sweet meets savory in a jubilant jamboree!\\"\\n\\n**Team No-Pineapple:** \\"No way, not in a million pizzas! Pineapple is a fruit that should be sunbathing on a tropical fruit platter, not freeloading on a sacred slice of pizza. The thought of warm, melty cheese conspiring with pineapple\'s sugary mischief is enough to make Italian ancestors turn in their tomato patches. Keep the pineapple in the fruit salad where it belongs!\\"\\n\\nAt the end of the day, whether pineapple belongs on pizza is a matter of personal taste. Some say it\'s a match made in heaven, others say it\'s a culinary crime. But let\'s face it, in the world of pizza toppings, where you can find everything from anchovies to zucchini, there\'s room for a little pineapple pizzazz. So, if your taste buds crave that sweet and tangy twist, go ahead and let your pineapple flag fly atop that glorious cheese-covered dough. Bon app\\u00e9tit, or as they say in pineapple paradise, \\"Aloha-ppetite!\\" \\ud83c\\udf4d\\ud83c\\udf55"]'
messages = [
    {"role": "system", "content": 
     '''
You are an AI judge evaluating responses from two different models to determine which one provides a better answer to a given prompt. 
Use the following step-by-step instructions to respond to user inputs.\n

Step 1 - The user will provide you with the prompt delimited by <prompt>, the model a response delimited by <model_a_response> and the model b response delimited by <model_b_Response>. Analyze the prompt. \n

Step 2 - Compare the two answer and look for similarity and difference. For the difference choose which model is wrong. \n

Step 3 - Decide which model as a more conscice, factual and pedagogial answer. Give the result in JSON format while explaining why you choose this model. If the 2 answer are similar you can answer "Tie". \n

Here is an example. 

Question : <prompt> What is an hexagone </prompt> \n
Model A response:
<model_a_response>
"An hexagone is a plane figure with six straight sides and angles."
</model_a_response> \n
 Model B response:
<model_b_response>
""An hexagone is a plane figure with seven straight sides and angles."
</model_b_response> \n

Answer: {
    Answer : "A", 
    Explication : "Model b is giving a false information by explaning that an hexagone has seven sides
}
     '''},

{"role": "user", "content": 
     f'''
User prompt:
<prompt>
{prompt}
</prompt> \n

Model A response:
<model_a_response>
"{model_a_response}"
</model_a_response> \n

Model B response:
<model_b_response>
"{model_b_response}"
</model_b_response> \n
   

     '''
    }

,
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


c:\Users\Yl\miniconda3\envs\cuda_test\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'role': 'assistant', 'content': 'Alright, I\'m trying to figure out which model provided the better answer to the prompt. The prompt was asking whether pineapple belongs on pizza, and both models gave different responses.\n\nModel A\'s answer was pretty detailed, explaining the debate between pineapple on pizza and pineapple on a salad. It even included a fun example with Hawaiian pizza. That seems to cover a lot of ground and show a good understanding of the topic.\n\nModel B, on the other hand, was more straightforward. It directly answered the question with a simple "No, pineapple shouldn\'t be on pizza." No extra fluff or arguments. It\'s concise and to the point.\n\nComparing both, Model A provided a more comprehensive and entertaining answer, while Model B gave a clear and direct response. Since the prompt was about a debate, Model A\'s detailed explanation might be the better choice as it offers a balanced view and even a humorous take, which can be engaging.\n</think>\n\nAnswe

In [2]:
def question(prompt,model_a_response,model_b_response):
    content_system = '''
    You are an AI judge evaluating responses from two different models to determine which one provides a better answer to a given prompt. 
    Use the following step-by-step instructions to respond to user inputs.\n

    Step 1 - The user will provide you with the prompt delimited by <prompt>, the model a response delimited by <model_a_response> and the model b response delimited by <model_b_Response>. Analyze the prompt. \n

    Step 2 - Compare the two answer and look for similarity and difference. For the difference choose which model is wrong. \n

    Step 3 - Decide which model as a more conscice, factual and pedagogial answer. Give the result in JSON format while explaining why you choose this model. If the 2 answer are similar you can answer "Tie". \n

    Here is an example. 
    Question : <prompt> What is an hexagone </prompt> \n
    Model A response:
    <model_a_response>
    "An hexagone is a plane figure with six straight sides and angles."
    </model_a_response> \n
    Model B response:
    <model_b_response>
    ""An hexagone is a plane figure with seven straight sides and angles."
    </model_b_response> \n
    Answer: {
        Answer : "A", 
        Explication : "Model b is giving a false information by explaning that an hexagone has seven sides
    }
    '''
    content_user = f'''
    User prompt:
    <prompt>
    {prompt}
    </prompt> \n
    Model A response:
    <model_a_response>
    "{model_a_response}"
    </model_a_response> \n
    Model B response:
    <model_b_response>
    "{model_b_response}"
    </model_b_response> \n
     '''
    return content_system,content_user

In [ ]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("data/train.csv")
sample = df.head(5)
serie = sample[['prompt','response_a','response_b']].iloc[0] 
prompt = serie.loc['prompt']
response_a = serie.loc['response_a']
response_b = serie.loc['response_b']
a,b = question(prompt,response_a,response_b)
print(a)
print(b)


    You are an AI judge evaluating responses from two different models to determine which one provides a better answer to a given prompt. 
    Use the following step-by-step instructions to respond to user inputs.


    Step 1 - The user will provide you with the prompt delimited by <prompt>, the model a response delimited by <model_a_response> and the model b response delimited by <model_b_Response>. Analyze the prompt. 


    Step 2 - Compare the two answer and look for similarity and difference. For the difference choose which model is wrong. 


    Step 3 - Decide which model as a more conscice, factual and pedagogial answer. Give the result in JSON format while explaining why you choose this model. If the 2 answer are similar you can answer "Tie". 


    Here is an example. 
    Question : <prompt> What is an hexagone </prompt> 

    Model A response:
    <model_a_response>
    "An hexagone is a plane figure with six straight sides and angles."
    </model_a_response> 

    Model

In [9]:
from openai import OpenAI
import os 

openai_api_key = os.environ.get('OPENAI_API_KEY')
client = OpenAI(api_key = openai_api_key)
list = []
def inference_4o_api(prompt,model_a_response,model_b_response):
    content_system,content_user = question(prompt,model_a_response,model_b_response)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": content_system},
            {
                "role": "user",
                "content": content_user
            }
        ]
    )
    list.append(completion)


inference_4o_api(prompt,response_a,response_b)

list

[ChatCompletion(id='chatcmpl-B7ngOX3m81JdDz8IJqjjI38Yoek3h', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "Answer": "A",\n    "Explication": "Model A provides a comprehensive, detailed analysis of the moral question regarding gender quotas in managerial positions and presents valid arguments for both sides. Furthermore, it adds a fun, engaging answer to the pizza question with humor and creativity. In contrast, Model B is more concise and does not expand on the points made for each perspective as thoroughly as Model A. It also lacks the playful tone that makes the discussion about pineapple on pizza more engaging. Therefore, Model A delivers a better and more pedagogical response overall."\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1741198976, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=Compl

In [21]:
print("%s" %list[0].choices[0].message.content)

{
    "Answer": "A",
    "Explication": "Model A provides a comprehensive, detailed analysis of the moral question regarding gender quotas in managerial positions and presents valid arguments for both sides. Furthermore, it adds a fun, engaging answer to the pizza question with humor and creativity. In contrast, Model B is more concise and does not expand on the points made for each perspective as thoroughly as Model A. It also lacks the playful tone that makes the discussion about pineapple on pizza more engaging. Therefore, Model A delivers a better and more pedagogical response overall."
}
